<a href="https://colab.research.google.com/github/CP2J/cp2j/blob/ACJ-14-Hybrid_Model/RecSys_Hybrid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 하이브리드 추천시스템 (CF_KNN + MF_SGD)

In [1]:
import numpy as np
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')
#rating = pd.read_csv('/content/drive/MyDrive/ml-100k/u.data', sep='\t', header=None, names=['user_id', 'item_id', 'rating', 'timestamp'])

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# 데이터 읽어 오기 
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('/content/drive/MyDrive/ml-100k/u.data', names=r_cols,  sep='\t',encoding='latin-1')
ratings = ratings[['user_id', 'movie_id', 'rating']].astype(int)            # timestamp 제거

In [4]:
# train test 분리
from sklearn.utils import shuffle
TRAIN_SIZE = 0.75
ratings = shuffle(ratings, random_state=1)
cutoff = int(TRAIN_SIZE * len(ratings))
ratings_train = ratings.iloc[:cutoff]
ratings_test = ratings.iloc[cutoff:]

In [5]:
# 정확도(RMSE)를 계산하는 함수 
def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

# CF-KNN

In [6]:
##### CF_KNN 추천 알고리즘 >>>>>>>>>>>>>>>
# "유저별 영화 평점"을 알아보기 위해 pivot table 활용, 결측치 처리
# train 셋을 full matrix로 변환
rating_matrix = ratings_train.pivot_table(values = 'rating', index = 'user_id', columns = 'movie_id')

# 유저들의 영화 평점 데이터프레임의 코사인유사도를 계산
from sklearn.metrics.pairwise import cosine_similarity
matrix_dummy = rating_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity, index = rating_matrix.index, columns = rating_matrix.index)

In [7]:
# 모델 평가 함수(RMSE)
def score_2(model, neighbor_size=0):
    id_pairs = zip(ratings_train['user_id'], ratings_train['movie_id'])
    y_pred = np.array([model(user, movie, neighbor_size) for (user, movie) in id_pairs])
    y_true = np.array(ratings_train['rating'])
    return RMSE(y_true, y_pred)
    # neighbor_size 입력받지 못하면 score_2 함수는 기본값 0으로 진행하고, cf_knn은 모든 사용자의 가중평균으로 예측
    # neighbor_size 입력된 경우에도, 해당 영화를 평가한 유저의 수가 K 보다 적을 수 있으므로 한번 더 확인 해야함

# CF + KNN
def cf_knn(user_id, movie_id, neighbor_size=0):
    # checks if the column with the name "movie_id" is present in the DataFrame "rating_matrix"
    if movie_id in rating_matrix:
        # 현재 사용자와 다른 사용자 간의 similarity 가져오기 (총 943개) : 'Series'
        sim_scores = user_similarity[user_id]
        
        # 현재 영화에 대한 모든 사용자의 rating값 가져오기 : 'Series'
        this_movie_ratings = rating_matrix[movie_id]
        
        # 현재 영화를 평가하지 않은 사용자의 index 가져오기
        none_rating_idx = rating_matrix[movie_id][rating_matrix[movie_id].isnull()].index
        
        # 현재 영화를 평가하지 않은 사용자 제거
        this_movie_ratings = this_movie_ratings.drop(none_rating_idx)
        
        # 현재 영화를 평가하지 않은 사용자의 similarity값 제거 -> 평가한 사람들만의 유사도 점수가 됨
        sim_scores = sim_scores.drop(none_rating_idx)

        ### KNN 추가 부분

        # neighbor_size 가 지정되지 않은 경우(나머지 전체 사용자 활용)
        if neighbor_size == 0:
            # 현재 영화를 평가한 모든 사용자의 가중평균 구하기
            mean_rating = np.dot(sim_scores, this_movie_ratings) / sim_scores.sum()

        # neighbor_size 가 지정된 경우
        else:
            # 지정된 neighbor_size와 해당 영화를 평가한 총 사용자수 중 작은 것으로 neighbor_size 결정
            neighbor_size = min( neighbor_size, len(sim_scores) )

            # array로 바꾸기 (argsort 사용하기 위해)
            sim_scores = np.array(sim_scores)
            this_movie_ratings = np.array(this_movie_ratings) # 오타... rating matrix를 어레이로 바꾸는게 아닌 현재 영화 평점을 바꾸는 것

            # 요소 크기 오름차순 정렬한 인덱스(user_id) 배열 반환 : argsort - Returns: index_array
            # https://codetorial.net/tips_and_examples/numpy_argsort.html
            user_idx = np.argsort(sim_scores)

            # 오름차순 정렬한 User 유사도 배열을 큰 것부터(뒤에서) neighbor_size 만큼 받기
            sim_scores = sim_scores[user_idx][-neighbor_size:]

            # 영화 Rating을 neighbor_size만큼 받기
            this_movie_ratings = this_movie_ratings[user_idx][-neighbor_size:]

            # 최종 예측값 계산
            mean_rating = np.dot( sim_scores, this_movie_ratings ) / sim_scores.sum()
    else:
        # movie_id가 rating_matrix의 칼럼값에 없는 경우
        mean_rating = 3.0
        
    return mean_rating

In [8]:
#최적의 K값 확인 
for k in (10,20,30,40,50):
    print('k = ', k,'RMSE = ', score_2(cf_knn, k))

k =  10 RMSE =  0.7586709162900331
k =  20 RMSE =  0.8411380746934589
k =  30 RMSE =  0.8722816313119262
k =  40 RMSE =  0.8896266842811573
k =  50 RMSE =  0.9001864275830908


# MF-SGD

In [9]:
##### MF-SGD 추천 알고리즘 >>>>>>>>>>>>>>>
class NEW_MF():
    def __init__(self, ratings, K, alpha, beta, iterations, verbose=True):  # 클래스 생성시 실행되는 초기화 함수
        self.R = np.array(ratings)    # df ratings를 np.array로 바꿔 self.R에 저장

        item_id_index = []    # 변수 초기화
        index_item_id = []    # 변수 초기화
        for i, one_id in enumerate(ratings):        # df ratings의 각 items에 대해서 아래 작업 수행
            item_id_index.append([one_id, i])       # save id - index 
            index_item_id.append([i, one_id])       # save index - id
        self.item_id_index = dict(item_id_index)    # dict로 변환
        self.index_item_id = dict(index_item_id)    # dict로 변환

        user_id_index = []                          # 사용자에 대해서도 실행
        index_user_id = []
        for i, one_id in enumerate(ratings.T):
            user_id_index.append([one_id, i])
            index_user_id.append([i, one_id])
        self.user_id_index = dict(user_id_index)    # dict로 변환
        self.index_user_id = dict(index_user_id)

        self.num_users, self.num_items = np.shape(self.R)
        self.K = K                                  # 잠재요인 수
        self.alpha = alpha                          # 학습률
        self.beta = beta                            # 정규화계수
        self.iterations = iterations                # SGD 계산 반복 횟수
        self.verbose = verbose                      # 중간 학습과정 출력여부

    # train set의 RMSE 계산
    def rmse(self):                            # 현재 P,Q로 RMSE 계산
        xs, ys = self.R.nonzero()              # R에서 평점이 있는(NOT NULL) 요소의 인덱스들
        self.predictions = []
        self.errors = []
        for x, y in zip(xs, ys):                            # 평점이 있는 요소(사용자 x, 아이템 y)에 대해:
            prediction = self.get_prediction(x, y)
            self.predictions.append(prediction)             # 예측값을 예측리스트에,
            self.errors.append(self.R[x, y] - prediction)   # 오차를 오차리스트에 저장
        self.predictions = np.array(self.predictions)
        self.errors = np.array(self.errors)
        return np.sqrt(np.mean(self.errors**2))

    # Predict Ratings for user i and item j
    def get_prediction(self, i, j):
        prediction = self.b + self.b_u[i] + self.b_d[j] + self.P[i, :].dot(self.Q[j, :].T)    # 평점 예측치(r^)의 식에 사용자, 아이템 편향 추가한 식
        # b : 전체평균, b_u[i] : 사용자i 평가경향(bias), b_d[j] : 아이템j 평가경향, 넷째항 : P, Q.T를 내적한 예측평점
        return prediction

    # Stochastic gradient descent to get optimized P and Q matrix
    def sgd(self):
        for i, j, r in self.samples:                # samples의 (user-item-rating) set에 대해 sgd적용
            prediction = self.get_prediction(i, j)
            e = (r - prediction)                    # 오차 = 실제평점 - 예측

            self.b_u[i] += self.alpha * (e - self.beta * self.b_u[i])     # 사용자,아이템의 경향성을 고려된 평점예측식의 편미분식으로 사용자 평가경향 업데이트
            self.b_d[j] += self.alpha * (e - self.beta * self.b_d[j])     # 아이템 평가경향 업데이트

            self.P[i, :] += self.alpha * (e * self.Q[j, :] - self.beta * self.P[i,:])   # 정규화항 추가된 평점예측식의 편미분식으로 P행렬 업뎃
            self.Q[j, :] += self.alpha * (e * self.P[i, :] - self.beta * self.Q[j,:])   # Q 업뎃

    # Test set을 선정 - 분리된 test set을 넘겨받아서 클래스 내부의 test set을 만드는 함수
    def set_test(self, ratings_test):
        test_set = []
        for i in range(len(ratings_test)):
            x = self.user_id_index[ratings_test.iloc[i, 0]]   # 현재 사용자의 인덱스를 user_id_index(매핑 리스트)에서 받아옴
            y = self.item_id_index[ratings_test.iloc[i, 1]]   # 현재 아이템의 인덱스를 item_id_index(매핑 리스트)에서 받아옴
            z = ratings_test.iloc[i, 2]                       # 현재 (사용자-아이템)의 평점
            test_set.append([x, y, z])          # 현재 (사용자-아이템-평점)을 test_set에 추가
            self.R[x, y] = 0                    # Setting test set ratings to 0 : MF는 R 전체를 사용해서 학습하기 때문에 test set은 평점을 지움
        self.test_set = test_set                # test_set을 클래스에 저장한다.
        return test_set                         # Return test set

    # Test set의 RMSE 계산
    def test_rmse(self):
        error = 0
        for one_set in self.test_set:
            predicted = self.get_prediction(one_set[0], one_set[1])
            error += pow(one_set[2] - predicted, 2)       # 오차(평점 실제값 - 예측평점)의 제곱을 누적한 값이 저장된다.
        return np.sqrt(error/len(self.test_set))          # error를 RMSE로 변환해서 돌려준다.

    # Training 하면서 test set의 정확도를 계산
    def test(self):   # MF모델을 SGD방식으로 훈련하는 핵심 함수이다.
        # Initializing user-feature and item-feature matrix
        self.P = np.random.normal(scale=1./self.K, size=(self.num_users, self.K))   # P행렬을 (평균 0, 표준편차 1/K인 정규분포 난수)로 초기화한다.
        self.Q = np.random.normal(scale=1./self.K, size=(self.num_items, self.K))   # Q행렬 초기화한다.

        # Initializing the bias terms
        self.b_u = np.zeros(self.num_users)           # 사용자 평가경향을 초기화, array크기는 사용자수(num_users)와 동일하다.
        self.b_d = np.zeros(self.num_items)           # 아이템 평가경향을 초기화한다.
        self.b = np.mean(self.R[self.R.nonzero()])    # 전체 평균을 구해서 저장한다.

        # List of training samples
        rows, columns = self.R.nonzero()              # 평점행렬 R 중에서 평점있는 요소의 인덱스를 가져온다.
        self.samples = [(i, j, self.R[i,j]) for i, j in zip(rows, columns)]   
        # SGD를 적용할 대상, 즉 평점이 있는 요소의 인덱스와 평점을 리스트로 만들어 samples에 저장한다.

        # Stochastic gradient descent for given number of iterations
        training_process = []                             # training_process 를 초기화한다. SGD를 한번 실행할 때마다 RMSE가 얼마나 개선되는지를 기록한다.
        for i in range(self.iterations):                  # 지정된 반복 횟수만큼 SGD를 실행한다.
            np.random.shuffle(self.samples)               # samples 를 임의로 섞는다. 출발점에 따라 수렴의 속도가 달라질 수 있기 때문이다.
            self.sgd()                                    # SGD를 실행하는 함수를 호출한다.
            rmse1 = self.rmse()                           # SGD로 P, Q, bu, bd가 업데이트 되었으므로 이에 따른 RMSE를 계산한다.
            rmse2 = self.test_rmse()                      # test set은 별도로 계산한다.
            training_process.append((i+1, rmse1, rmse2))  # 결과를 저장한다.
            if self.verbose:                              # verbose가 True이면 10회 반복마다 중간 결과를 표시한다.
                if (i+1) % 10 == 0:
                    print("Iteration: %d ; Train RMSE = %.4f ; Test RMSE = %.4f" % (i+1, rmse1, rmse2))
        return training_process

    # Ratings for given user_id and item_id
    def get_one_prediction(self, user_id, item_id):
        prediction = self.get_prediction(self.user_id_index[user_id], self.item_id_index[item_id])
        return prediction

    # Full user-movie rating matrix (모든 사용자의 모든 아이템에 대한 예측치(Full matrix)를 계산해서 돌려준다)
    def full_prediction(self):
        return self.b + self.b_u[:,np.newaxis] + self.b_d[np.newaxis,:] + self.P.dot(self.Q.T)
        # $\hat{r}_{ij}=b+bu_i+bd_j+\sum_{k=1}^Kp_{ik}q_{kj}$


In [10]:
# MF클래스 생성 및 학습
R_temp = ratings.pivot(index='user_id', columns='movie_id', values='rating').fillna(0)    # 전체 데이터를 full matrix로 변환한다.
mf = NEW_MF(R_temp, K=200, alpha=0.001, beta=0.02, iterations=250, verbose=True)          # NEW-MF 클래스를 생성한다.
test_set = mf.set_test(ratings_test)    # ratings_test를 테스트 데이터로 지정하도록 set_test() 함수를 호출한다.
result = mf.test()                      # 정해진 파라미터에 따라 MF 훈련과 정확도 계산을 실행한다.

Iteration: 10 ; Train RMSE = 0.9664 ; Test RMSE = 0.9834
Iteration: 20 ; Train RMSE = 0.9420 ; Test RMSE = 0.9644
Iteration: 30 ; Train RMSE = 0.9313 ; Test RMSE = 0.9566
Iteration: 40 ; Train RMSE = 0.9253 ; Test RMSE = 0.9524
Iteration: 50 ; Train RMSE = 0.9214 ; Test RMSE = 0.9497
Iteration: 60 ; Train RMSE = 0.9186 ; Test RMSE = 0.9480
Iteration: 70 ; Train RMSE = 0.9165 ; Test RMSE = 0.9468
Iteration: 80 ; Train RMSE = 0.9147 ; Test RMSE = 0.9459
Iteration: 90 ; Train RMSE = 0.9130 ; Test RMSE = 0.9451
Iteration: 100 ; Train RMSE = 0.9111 ; Test RMSE = 0.9444
Iteration: 110 ; Train RMSE = 0.9088 ; Test RMSE = 0.9435
Iteration: 120 ; Train RMSE = 0.9056 ; Test RMSE = 0.9423
Iteration: 130 ; Train RMSE = 0.9011 ; Test RMSE = 0.9406
Iteration: 140 ; Train RMSE = 0.8946 ; Test RMSE = 0.9382
Iteration: 150 ; Train RMSE = 0.8860 ; Test RMSE = 0.9350
Iteration: 160 ; Train RMSE = 0.8754 ; Test RMSE = 0.9313
Iteration: 170 ; Train RMSE = 0.8631 ; Test RMSE = 0.9277
Iteration: 180 ; Train 

In [11]:
# iteration = 200
# learning_rate = 0.01
# lmbda = 0.01

In [12]:
##### Hybrid 추천 알고리즘

def recommender0(recomm_list, mf):                        # MF 알고리즘의 예측값을 받아오는 함수이다. 
    recommendations = np.array([mf.get_one_prediction(user, movie) for (user, movie) in recomm_list])
    # 추천대상 리스트(recomm_list)의 항목 각각에 대해서 MF클래스의 get_one_prediction() 함수를 불러서 예측값을 받아온 후에 np.array로 변환한다.
    return recommendations

def recommender1(recomm_list, neighbor_size=0):           # CF 알고리즘의 예측값을 받아오는 함수이다.
    recommendations = np.array([cf_knn(user, movie, neighbor_size) for (user, movie) in recomm_list])
    # 추천대상 리스트(recomm_list)의 항목 각각에 대해서 cf_knn 함수로 예측값을 받아 np.array로 변환한다.
    return recommendations

recomm_list = np.array(ratings_test.iloc[:, [0, 1]])      # test set을 np.array형식의 추천 대상 리스트로 만든다.
predictions0 = recommender0(recomm_list, mf)              # MF기반 알고리즘의 예측값을 받아온다.
RMSE(ratings_test.iloc[:, 2], predictions0)               # MF기반 알고리즘의 RMSE를 계산한다.
predictions1 = recommender1(recomm_list, 40)              # CF기반 알고리즘의 예측값을 받아온다.
RMSE(ratings_test.iloc[:, 2], predictions1)               # CF기반 알고리즘의 RMSE를 계산한다.

weight = [0.8, 0.2]                                                 # 두 알고리즘의 결합 가중치를 지정한다.
predictions = predictions0 * weight[0] + predictions1 * weight[1]   # 가중치에 따라 두 추천 알고리즘에서 가져온 예측값을 가중평균한다.
RMSE(ratings_test.iloc[:, 2], predictions)                          # 하이브리드 모델의 RMSE를 계산한다.

0.9114388438903243

In [14]:
# 가중치를 0~1까지 0.01 간격으로 바꿔가면서 RMSE 계산
for i in np.arange(0, 1, 0.01):
    weight = [i, 1.0 - i]
    predictions = predictions0 * weight[0] + predictions1 * weight[1]
    print("Weights - %.2f : %.2f ; RMSE = %.7f" % (weight[0], 
           weight[1], RMSE(ratings_test.iloc[:, 2], predictions)))

Weights - 0.00 : 1.00 ; RMSE = 1.0131178
Weights - 0.01 : 0.99 ; RMSE = 1.0109799
Weights - 0.02 : 0.98 ; RMSE = 1.0088612
Weights - 0.03 : 0.97 ; RMSE = 1.0067617
Weights - 0.04 : 0.96 ; RMSE = 1.0046814
Weights - 0.05 : 0.95 ; RMSE = 1.0026206
Weights - 0.06 : 0.94 ; RMSE = 1.0005794
Weights - 0.07 : 0.93 ; RMSE = 0.9985578
Weights - 0.08 : 0.92 ; RMSE = 0.9965560
Weights - 0.09 : 0.91 ; RMSE = 0.9945742
Weights - 0.10 : 0.90 ; RMSE = 0.9926123
Weights - 0.11 : 0.89 ; RMSE = 0.9906706
Weights - 0.12 : 0.88 ; RMSE = 0.9887492
Weights - 0.13 : 0.87 ; RMSE = 0.9868482
Weights - 0.14 : 0.86 ; RMSE = 0.9849677
Weights - 0.15 : 0.85 ; RMSE = 0.9831078
Weights - 0.16 : 0.84 ; RMSE = 0.9812686
Weights - 0.17 : 0.83 ; RMSE = 0.9794503
Weights - 0.18 : 0.82 ; RMSE = 0.9776530
Weights - 0.19 : 0.81 ; RMSE = 0.9758767
Weights - 0.20 : 0.80 ; RMSE = 0.9741217
Weights - 0.21 : 0.79 ; RMSE = 0.9723880
Weights - 0.22 : 0.78 ; RMSE = 0.9706757
Weights - 0.23 : 0.77 ; RMSE = 0.9689849
Weights - 0.24 :

- 베스트 : Weights - 0.91 : 0.09 ; RMSE = 0.9097518


                    (MF-SGD 91%, CF-KNN 9%)